In [174]:
# https://huggingface.co/docs/transformers/training
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [175]:
import torch 
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os

import torch.nn as nn
import torch.nn.functional as F

In [176]:
# PyTorch models inherit from torch.nn.Module
class SentenceMultiClassClassifier(nn.Module):
    def __init__(self,number_class, pretrained_model):
        super(SentenceMultiClassClassifier, self).__init__()
        self.number_class = number_class
        #self.pretrained = AutoModel.from_pretrained(pretrained_model).to(device)
        self.pretrained = AutoModel.from_pretrained(pretrained_model,config=AutoConfig.from_pretrained(pretrained_model, output_attentions=True,output_hidden_states=True))

        #self.dropout = nn.Dropout(0.5) 
        #self.fc1 = nn.Linear(768, 1200)
        #self.fc2 = nn.Linear(1200, 1400)
        #self.fc3 = nn.Linear(1400, number_class)
        
        self.linear = nn.Linear(768, number_class)
        self.layeroutput = torch.nn.Sigmoid()



    def forward(self, input_ids, token_type_ids, attention_mask):            
        output_pretrained = self.pretrained(input_ids, token_type_ids, attention_mask)
        # Freeze the BERT parameters
        for param in self.pretrained.parameters():
            param.requires_grad = False
            
        #x = F.relu(self.fc1(output_pretrained.last_hidden_state[:,0,:].view(-1,768)))
        #x = self.dropout(x)
        #x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        
        x = output_pretrained.last_hidden_state[:,0,:].view(-1,768)
        x = self.linear(x)
        x = self.layeroutput(x)
        return x

In [177]:
TOKENIZER_MODEL = "bert-base-multilingual-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model = "bert-base-multilingual-uncased"

In [178]:
def model_fn(model_dir):   
    NUM_CLASS = 11
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_MODEL)
    model = SentenceMultiClassClassifier(NUM_CLASS,TOKENIZER_MODEL)
    with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
        #model = torch.jit.load(f)
        model.load_state_dict(torch.load(f))

    model.eval()
    return model.to(device), tokenizer

In [179]:
def predict_fn(text, model_and_tokenizer):
    # destruct model and tokenizer
    model, tokenizer = model_and_tokenizer    
    
    # Tokenize sentences
    tokenized_text = tokenizer(text, max_length = 128, padding="max_length", return_tensors="pt")
    tokenized_text['input_ids'] = tokenized_text['input_ids'].to(device)
    tokenized_text['token_type_ids'] = tokenized_text['token_type_ids'].to(device)
    tokenized_text['attention_mask'] = tokenized_text['attention_mask'].to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**tokenized_text)

    # return dictonary, which will be json serializable
    return model_output.tolist()

In [180]:
model_and_tokenizer = model_fn("./model")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [181]:
text = "I am so happy about this situation"

In [182]:
results = predict_fn(text, model_and_tokenizer)
results

[[0.38515618443489075,
  0.08222470432519913,
  0.30914536118507385,
  0.11874663084745407,
  0.21496078372001648,
  0.04816637188196182,
  0.19921928644180298,
  0.21099203824996948,
  0.343722939491272,
  0.018559658899903297,
  0.014233306050300598]]

## Evaluation

In [183]:
import pandas as pd 
import numpy as np

In [184]:
df = pd.read_csv("./dataset/sem_eval_2018_task_1_test.csv")

In [185]:
df.head(5)

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-En-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,True,True,False,False,False,False,True,False,False,False,True
1,2018-En-03739,"Academy of Sciences, eschews the normally sobe...",False,False,True,False,False,False,False,False,False,False,False
2,2018-En-00385,I blew that opportunity -__- #mad,True,False,True,False,False,False,False,False,True,False,False
3,2018-En-03001,This time in 2 weeks I will be 30... 😥,False,False,False,False,True,False,False,False,True,False,False
4,2018-En-01988,#Deppression is real. Partners w/ #depressed p...,False,False,False,True,False,False,False,False,True,False,False


In [200]:
label_name = [label for label in df.columns if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(label_name)}
id2label

{0: 'anger',
 1: 'anticipation',
 2: 'disgust',
 3: 'fear',
 4: 'joy',
 5: 'love',
 6: 'optimism',
 7: 'pessimism',
 8: 'sadness',
 9: 'surprise',
 10: 'trust'}

In [187]:
text_tweet = df.loc[:,["Tweet"]].values.tolist()

In [188]:
text_tweet[0]

['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell']

In [189]:
results = predict_fn(text_tweet[0], model_and_tokenizer)
results

[[0.49910658597946167,
  0.11445894092321396,
  0.5174931883811951,
  0.1451595574617386,
  0.27547383308410645,
  0.0479339063167572,
  0.2813677191734314,
  0.08886060118675232,
  0.22894766926765442,
  0.027332641184329987,
  0.036866359412670135]]

In [190]:
def proba_to_label(prediction_proba,id2label,threshold):
    prediction_label = []
    for index in range(len(prediction_proba)):
        if prediction_proba[index] > threshold:
            prediction_label.append(id2label[index])
            
    return prediction_label

In [191]:
print(proba_to_label(results[0],id2label,0.3))

['anger', 'disgust']


In [192]:
label_index = df.iloc[:,2:].astype(int)
label_index.head(5)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,1,1,0,0,0,0,1,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,1,0,0


In [193]:
label_index = label_index.to_numpy()
label_index

array([[1, 1, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [194]:
predictions_proba = np.zeros_like(label_index,dtype=float)

In [195]:
for index in range(len(text_tweet)):
    result = predict_fn(text_tweet[index], model_and_tokenizer)
    result = np.squeeze(np.array(result), axis=0)    
    predictions_proba[index,:] = result    

In [196]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [197]:
def multi_label_metrics(labels, predictions_proba, threshold = 0.5):
    y_true = labels

    predictions = np.zeros_like(labels)
    predictions[np.where(predictions_proba >= threshold)] = 1
    
    y_pred = predictions
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1_micro_average = f1_score(y_true, y_pred, average='micro')
    
    metrics = {'precision': precision,
               'recall': recall,
               'f1': f1_micro_average}
    return metrics

In [198]:
metrics = multi_label_metrics(label_index, predictions_proba, threshold = 0.3)

In [199]:
metrics

{'precision': 0.4748230950671726,
 'recall': 0.5883848011183124,
 'f1': 0.5255391600454029}